In [2]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM

In [3]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data

data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')

In [4]:
def setup_data(data):
    """simplify setting up data correctly"""
    #set up x and y
    y = data[['Inside Good Share','Outside Good Share']]
    x =  data[['Network Score','Satisfaction Score','PPO','Premium']]
    return x,y


def setup_hausman(data):
    #calculate hausmann insturments
    price = data['Premium']
    
    mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
    plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
    exog = np.array( data[['Network Score','Satisfaction Score', 'PPO']])
    
    #calc avg price in other markets
    hausman_mod1 = sm.OLS(price, sm.add_constant(plan_dum))
    hausman_fit1 = hausman_mod1.fit()
    hausman_instr1 = hausman_fit1.fittedvalues
    hausman_instr1 = np.array([hausman_instr1]).transpose()
    
    #avg price in same mkt
    hausman_mod2 = sm.OLS(price, sm.add_constant(mkt_dum))
    hausman_fit2 = hausman_mod2.fit()
    hausman_instr2 = hausman_fit2.fittedvalues
    hausman_instr2 = np.array([hausman_instr2]).transpose()
    
    #concat hausman instr with exog variables
    instr = np.concatenate( (exog, hausman_instr1, hausman_instr2), axis =1 )
    
    return instr

x,y =  setup_data(data)
z = setup_hausman(data)

In [5]:
#set up useful global variables 
NMKTS = data['Market_ID'].nunique()
NPLANS = data['Plan_ID'].nunique()
NOBS = data['Plan_ID'].count()
NSIM = 20

theta1 = np.array([3.5,2,1,-2])
theta2 = np.array([.15,.1,.5]) # initialize theta2 for testing purposes
delta = np.ones(NOBS)*(-2)

#print global variables
print NMKTS,NPLANS,NOBS

600 16 3300


In [6]:
#set up random draws v
v = np.random.normal(size=(NSIM,3)) #same simulation for all markets
#np.savetxt("simulations.csv", v, delimiter=",")

#use same simulations each time
v = np.genfromtxt('simulations.csv', delimiter=',')
v = np.tile(v.reshape(NSIM,3,1) , (1,1,NOBS))

# Part 1 - Estimation

## Calculating $\delta_{jt}$, $\xi_{jt}$

In [7]:
def cal_sim_s(data, v, delta, theta2):
    """calculate market share for each simulated consumer"""
    
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO']]).transpose()
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1,3300))
    delta  = np.tile( delta  ,(NSIM,1))
    
    #add to calcualte market shares
    sim_exp = pd.DataFrame( np.exp(delta + (theta2*v*x).sum(axis=1)).transpose() ) 
    
    #sum up between markets
    sim_exp['mkt_id'] = data['Market_ID']
    sum_exp = sim_exp.groupby('mkt_id').sum() 
    sum_exp = pd.merge(data.copy()[['Market_ID']], sum_exp, 
                       right_index=True, left_on = 'Market_ID')
    
    #format so I can broadcast
    sim_exp = np.array(sim_exp).transpose()[:-1]
    sum_exp = np.array(sum_exp).transpose()[1:] + 1
    
    return sim_exp/sum_exp


def cal_s(data, v, delta, theta2):
    """Calculate market share
    Calculates individual choice probability first, then take sum"""
    
    shares = (1./NSIM)*cal_sim_s(data, v, delta, theta2).sum(axis=0)
    return shares


s = cal_s(data, v, delta, theta2)

In [8]:
def cal_delta(data, v, theta2, error = 1e-3, maxiter = 500):
    """Calculate mean utility via contraction mapping"""

    niter = 0
    
    #initialize loop parameters
    delta = np.zeros(NOBS)
    s = cal_s(data, v, delta, theta2)
    diff = np.log(data['Inside Good Share']) - np.log(s)
    
    
    while ((abs(diff).max() > 1e-6) #this is easier to converge
           and (abs(diff).mean() > error) 
           and niter < maxiter):
        
        s = cal_s(data, v, delta, theta2)
        diff = np.log(data['Inside Good Share']) - np.log(s)

        if np.isnan(diff).sum():
            raise Exception('nan in diffs')
            
        delta += diff
        niter += 1

    return delta


delta = cal_delta(data, v, theta2)
print delta.shape
print delta.mean()

(3300,)
0.0537547330113


In [9]:
#initialize theta1 i.e. alpha and beta
def cal_xi(data, delta, theta1):
    """Calculate xi with F.O.C"""
    x,y =  setup_data(data)
    
    xi = np.matmul(np.array(x),theta1)
    
    xi = delta - np.matmul(np.array(x),theta1)
    
    return  xi

xi = cal_xi(data, delta,  theta1)
print xi.max(), xi.min(), xi.mean()

0.37984154348916377 -0.7035693264545015 -0.243041055595


## Calculating  $\theta_1,\theta_2$

where $\theta_1 = (\alpha, \beta)$

I only solve GMM over $\theta_2$, the non-linear parameters

In [10]:
def cal_theta1(data, delta, z, T):
    """  calculate theta 1 using FOCs (X1'Z T Z'X )^-1 X1'Z T Z' delta """
    
    #set up variables
    x,y =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    
    #build up to main equation
    XtZ = X.transpose().dot(Z)
    ZtX = Z.transpose().dot(X)
        
    first_exp = np.linalg.inv( XtZ.dot(T).dot(ZtX))
    second_exp = XtZ.dot(T).dot(Z.transpose()).dot(delta)
    theta1 = first_exp.dot(second_exp)
    return theta1

T =  np.linalg.inv( z.transpose().dot(z) )
print cal_theta1(data, delta, z, T)

[ 2.65526242  1.96570924  0.67239952 -1.72223237]


In [11]:
def gmm_objective(theta2_init, data,  v, z,  T):
    """w' z phi z' w, of theta2"""
    #set up variables
    x,y =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    
    #do calculations
    delta = cal_delta(data, v, theta2_init)
    theta1 = cal_theta1(data, delta, z, T)
    xi = cal_xi(data, delta, theta1)
        
    xitZ = xi.transpose().dot(Z)
    Ztxi = Z.transpose().dot(xi)
    return xitZ.dot(T).dot(Ztxi)
    

def calc_theta2(data, v, z, T, theta2_init,NM=True):
    """calculate theta2 using scipy"""
    if NM:
        theta2 = minimize(gmm_objective, theta2_init, args=(data,  v, z, T), method='Nelder-Mead',
                      options={'maxiter':1000, 'disp': True, 'xatol':1e-4, 'fatol':1})
    else:
        theta2 = minimize(gmm_objective, theta2_init, args=(data,  v, z, T), method='BFGS',
                      options={'maxiter':100, 'disp': True})
    return theta2.x



#run some tests
#theta2_init = np.array([1,.5,2])
#theta2_test = calc_theta2( data, v, T, theta2_init)
# print theta2_test


theta2_init = np.array([1,1,1])
calc_theta2(data, v, z, T, theta2_init)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 44
         Function evaluations: 88


array([1.38684589, 1.21411172, 0.51646302])

In [12]:
theta2_init = np.array([1,1,1])

def calc_theta(data, v, theta2_init):
    
    #initialize theta
    x,y =  setup_data(data)
    z = setup_hausman(data)
    
    X,Z = np.array(x), np.array(z)
    theta2 = theta2_init 
    
    #on first step, use consistent approximation of T
    T =  np.linalg.inv( z.transpose().dot(z) )
    for i in range(2):  
       
        #on second use T using estimated xi
        if i==1:
            xi = cal_xi(data, delta, theta1)
            xi =np.array([xi]).transpose()
            T =  np.linalg.inv( Z.transpose().dot(xi).dot(xi.transpose()).dot(Z) )
        
        theta2 = calc_theta2(data, v, z, T, theta2)
        delta = cal_delta(data, v, theta2)
        theta1 = cal_theta1(data, delta, z, T)
        
    return theta1, theta2

theta = calc_theta(data, v, theta2_init)


print theta

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 44
         Function evaluations: 88
Optimization terminated successfully.
         Current function value: -2363861164543129600.000000
         Iterations: 125
         Function evaluations: 336
(array([ 3.34952545,  1.95632696,  0.6506052 , -1.5642128 ]), array([1.43279579, 1.25336653, 0.50565619]))


In [13]:
#save xi and write to array for counterfactuals
theta1_est, theta2_est = theta

delta_est = cal_delta(data, v, theta2_est)
xi_est = cal_xi(data, delta_est, theta1_est)

np.savetxt("xi.csv", xi_est, delimiter=",")
#print data['Inside Good Share'] - cal_s(data, v, delta, theta2)

## Calculate Standard Errors